<a href="https://colab.research.google.com/github/sheldonkemper/bank_of_england/blob/main/notebooks/modelling/sk_gen_ai_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/sheldonkemper/bank_of_england/blob/main/notebooks/modelling/sk_gen_ai_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
===================================================
Author: Sheldon Kemper
Role: Data Engineering Lead, Bank of England Employer Project (Quant Collective)
LinkedIn: https://www.linkedin.com/in/sheldon-kemper
Date: 2025-02-04
Version: 1.2

Description:
    This notebook contains a class-based implementation of a Retrieval Augmented Generation (RAG) engine
    designed to analyze bank quarterly earnings call transcripts (in PDF format) stored on Google Drive.
    The code performs the following tasks:

    1. Configures an LLM pipeline using a Flan-T5-based model for text summarization.
    2. Sets up sentence-transformer based embeddings for document vectorization.
    3. Loads and splits PDF documents from one or more specified directories.
    4. Chunks the documents and builds a vector index using Chroma, persisting the index to Google Drive.
    5. Optionally loads an existing persisted vector index to avoid re-indexing, via the 'rebuild_index' parameter.
    6. Retrieves context relevant to user queries from the vector index with token truncation to enforce input limits.
    7. Maintains conversation memory for interactive sessions.
    8. Supports both interactive and programmatic prompt-based querying.
    9. Includes a 'test_mode' option for quick testing with a single PDF.
===================================================
"""

"\n===================================================\nAuthor: Sheldon Kemper\nRole: Data Engineering Lead, Bank of England Employer Project (Quant Collective)\nLinkedIn: https://www.linkedin.com/in/sheldon-kemper\nDate: 2025-02-04\nVersion: 1.2\n\nDescription:\n    This notebook contains a class-based implementation of a Retrieval Augmented Generation (RAG) engine\n    designed to analyze bank quarterly earnings call transcripts (in PDF format) stored on Google Drive.\n    The code performs the following tasks:\n\n    1. Configures an LLM pipeline using a Flan-T5-based model for text summarization.\n    2. Sets up sentence-transformer based embeddings for document vectorization.\n    3. Loads and splits PDF documents from one or more specified directories.\n    4. Chunks the documents and builds a vector index using Chroma, persisting the index to Google Drive.\n    5. Optionally loads an existing persisted vector index to avoid re-indexing, via the 'rebuild_index' parameter.\n    6.

In [2]:
# install langchain-community
!pip install -q langchain-community pypdf tiktoken chromadb sentence-transformers datasets rouge-score huggingface_hub torch transformers  > /dev/null 2>&1


In [3]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.memory import ConversationBufferWindowMemory
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain
from google.colab import drive
# For evaluation metrics (ROUGE)
from rouge_score import rouge_scorer

from huggingface_hub import hf_hub_download
import warnings
import os
from google.colab import userdata

In [4]:
warnings.filterwarnings("ignore", category=DeprecationWarning)
# Mount Google Drive to the root location with force_remount
drive.mount('/content/drive', force_remount=True)
userdata.get('HF')
os.environ["HF"] = userdata.get('HF') # Replace with your actual token

Mounted at /content/drive


# A class-based implementation of an LLM Retrieval Augmented Generation (RAG) engine

In [5]:
# import os
# import re
# import torch
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
# from datasets import Dataset
# from langchain.document_loaders import PyPDFLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.vectorstores import Chroma
# from langchain.memory import ConversationBufferWindowMemory
# from langchain_community.llms import HuggingFacePipeline
# from langchain.chains import ConversationalRetrievalChain
# from langchain.docstore.document import Document
# from langchain.agents import Tool, initialize_agent, AgentType

# # --- Configuration ---
# CONFIG = {
#     "pdf_folders": [
#         "/content/drive/MyDrive/BOE/bank_of_england/data/raw/jpmorgan",
#         "/content/drive/MyDrive/BOE/bank_of_england/data/raw/ubs"
#     ],
#     "persist_directory": "/content/drive/MyDrive/BOE/bank_of_england/data/model_outputs",
#     "llm_model_name": "google/bigbird-pegasus-large-arxiv",  # For long sequences
#     "embedding_model_name": "sentence-transformers/all-mpnet-base-v2",
#     "text_generation_pipeline_task": "text2text-generation",
#     "max_length": 1024,
#     "temperature": 0.1,
#     "top_p": 0.8,
#     "batch_size": 8,
#     "chunk_size": 1000,
#     "chunk_overlap": 100,
#     "chunk_threshold": 1024,
#     "memory_window_k": 10,
#     "retriever_search_k": 5
# }

# # --- Master Agent Prompt (Refined) ---
# MASTER_AGENT_PROMPT = (
#     "You are an expert in analyzing bank earnings call transcripts. "
#     "Your task is to answer questions accurately using provided tools.\n\n"
#     "When answering, follow EXACTLY this format:\n\n"
#     "Thought: Briefly explain your reasoning.\n"
#     "Action: Use one of the tools: JP_Morgan_RAG or UBS_RAG. "
#     "To use a tool, write: Tool_Name(\"query\"), for example: JP_Morgan_RAG(\"what is net income?\").\n"
#     "Observation: Report the result from the tool. Keep it concise.\n"
#     "Final Answer: Give a final, short answer to the original question.\n\n"
#     "Question: {input}\n"  # Place the question directly in the prompt for clarity
#     "Begin!"
# )


# # --- RAGChatbot Class ---
# class RAGChatbot:
#     """
#     A RAG chatbot that ingests PDF earnings call transcripts, builds a vector store,
#     and uses a ConversationalRetrievalChain for Q&A.
#     """
#     def __init__(self, config):
#         self.config = config
#         self.hf_token = os.environ.get('HF') # Access token from environment variable
#         self.pdf_folders = config["pdf_folders"]
#         self.persist_directory = config["persist_directory"]
#         self.max_length = config["max_length"]
#         self.batch_size = config["batch_size"]
#         self.chunk_size = config["chunk_size"]
#         self.chunk_overlap = config["chunk_overlap"]
#         self.chunk_threshold = config["chunk_threshold"]
#         self.memory_window_k = config["memory_window_k"]
#         self.retriever_search_k = config["retriever_search_k"]

#         self._setup_llm()
#         self._setup_embeddings()
#         self._load_documents()
#         self._build_vector_store()
#         self._build_summary_index()
#         self._setup_retrieval_chain()

#     def _setup_llm(self):
#         model_name = self.config["llm_model_name"]
#         self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=self.hf_token) # Use use_auth_token
#         self.model = AutoModelForSeq2SeqLM.from_pretrained(
#             model_name, torch_dtype=torch.float16, use_auth_token=self.hf_token) # Use use_auth_token
#         if self.model.config.pad_token_id is None:
#             self.model.config.pad_token_id = self.tokenizer.eos_token_id
#         if torch.cuda.is_available():
#             self.model.to("cuda")
#         # Use num_beams=1 to avoid beam search issues that trigger CUDA asserts.
#         self.pipe = pipeline(
#             self.config["text_generation_pipeline_task"],
#             model=self.model,
#             tokenizer=self.tokenizer,
#             max_length=self.max_length,
#             temperature=self.config["temperature"],
#             top_p=self.config["top_p"],
#             do_sample=True,
#             batch_size=self.batch_size,
#             num_beams=1,
#             pad_token_id=self.tokenizer.eos_token_id
#         )
#         self.llm = HuggingFacePipeline(pipeline=self.pipe)

#     def _setup_embeddings(self):
#         emb_model = self.config["embedding_model_name"]
#         self.embeddings = HuggingFaceEmbeddings(model_name=emb_model) # Removed token argument

#     def _load_documents(self):
#         self.documents = []
#         for folder in self.pdf_folders:
#             bank = os.path.basename(folder).lower()
#             files = [f for f in os.listdir(folder) if f.endswith(".pdf")]
#             for file in files:
#                 path = os.path.join(folder, file)
#                 try:
#                     loader = PyPDFLoader(path, extract_images=False)
#                     docs = loader.load_and_split()
#                     for doc in docs:
#                         doc.metadata["bank"] = bank
#                         doc.metadata["source_pdf"] = file
#                     self.documents.extend(docs)
#                     print(f"Loaded: {file} from {folder}")
#                 except Exception as e:
#                     print(f"Error loading {file}: {e}")

#     def _chunk_document(self, doc: Document) -> list[Document]:
#         tokens = self.tokenizer.encode(doc.page_content)
#         if len(tokens) > self.chunk_threshold:
#             splitter = RecursiveCharacterTextSplitter(
#                 chunk_size=self.chunk_size, chunk_overlap=self.chunk_overlap)
#             chunks = splitter.split_documents([doc])
#             return self._remove_duplicates(chunks)
#         return [doc]

#     @staticmethod
#     def _remove_duplicates(chunks: list[Document]) -> list[Document]:
#         seen = set()
#         unique = []
#         for chunk in chunks:
#             text = chunk.page_content.strip()
#             if text not in seen:
#                 seen.add(text)
#                 unique.append(chunk)
#         return unique

#     def _build_vector_store(self):
#         all_chunks = []
#         for doc in self.documents:
#             all_chunks.extend(self._chunk_document(doc))
#         self.raw_db = Chroma.from_documents(
#             all_chunks, embedding=self.embeddings, persist_directory=self.persist_directory)
#         self.raw_db.persist()
#         print(f"Built raw vector store with {len(all_chunks)} chunks.")

#     def _build_summary_index(self):
#         # For simplicity, we use the same chunks as summaries.
#         all_chunks = []
#         for doc in self.documents:
#             all_chunks.extend(self._chunk_document(doc))
#         self.summary_db = Chroma.from_documents(
#             all_chunks, embedding=self.embeddings,
#             persist_directory=os.path.join(self.persist_directory, "summaries"))
#         self.summary_db.persist()
#         print(f"Built summary vector index with {len(all_chunks)} chunks.")

#     def _setup_retrieval_chain(self):
#         memory = ConversationBufferWindowMemory(
#             k=self.memory_window_k, memory_key="chat_history", return_messages=True)
#         self.retrieval_chain = ConversationalRetrievalChain.from_llm(
#             llm=self.llm,
#             retriever=self.summary_db.as_retriever(search_kwargs={"k": self.retriever_search_k}),
#             memory=memory,
#             verbose=True
#         )

#     def answer_query(self, query: str) -> str:
#         response = self.retrieval_chain({"question": query})
#         return response.get("answer", "").strip()


# # --- Create Multi-Agent Instances ---
# # Filter PDF folders for each bank.
# jpm_folders = [folder for folder in CONFIG["pdf_folders"] if "jpmorgan" in folder.lower()]
# ubs_folders = [folder for folder in CONFIG["pdf_folders"] if "ubs" in folder.lower()]

# # Create separate configurations for each bank.
# CONFIG_JPM = CONFIG.copy()
# CONFIG_JPM["pdf_folders"] = jpm_folders

# CONFIG_UBS = CONFIG.copy()
# CONFIG_UBS["pdf_folders"] = ubs_folders

# # Initialize separate RAGChatbot instances.
# jpm_chatbot = RAGChatbot(CONFIG_JPM)
# ubs_chatbot = RAGChatbot(CONFIG_UBS)

# # --- Define Tools for Each Agent ---
# def jpm_tool(query: str) -> str:
#     return jpm_chatbot.answer_query(query)

# def ubs_tool(query: str) -> str:
#     return ubs_chatbot.answer_query(query)

# jpm_tool_instance = Tool(
#     name="JP_Morgan_RAG",
#     func=jpm_tool,
#     description="Answers questions about JP Morgan earnings call transcripts."
# )

# ubs_tool_instance = Tool(
#     name="UBS_RAG",
#     func=ubs_tool,
#     description="Answers questions about UBS earnings call transcripts."
# )

# # --- Master Agent Integration ---
# master_agent = initialize_agent(
#     [jpm_tool_instance, ubs_tool_instance],
#     jpm_chatbot.llm,  # Using the same LLM pipeline; both agents use similar config.
#     agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
#     verbose=True,
#     handle_parsing_errors=True,
#     agent_kwargs={"prefix": MASTER_AGENT_PROMPT} # Using the updated MASTER_AGENT_PROMPT
# )

# # --- Chatbot Loop (Master Agent) ---
# def run_master_agent():
#     print("Master Agent Chatbot (type 'exit' to quit)")
#     while True:
#         user_q = input("You: ")
#         if user_q.lower() == "exit":
#             print("Exiting Master Agent Chatbot. Goodbye!")
#             break
#         answer = master_agent.run(user_q)
#         print(f"\nMaster Agent Answer:\n{answer}\n")

# if __name__ == "__main__":
#     run_master_agent()

In [ ]:
import os
import re
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from datasets import Dataset
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.memory import ConversationBufferWindowMemory
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain
from langchain.docstore.document import Document
from langchain.agents import Tool, initialize_agent, AgentType

# --- Configuration ---
CONFIG = {
    "pdf_folders": [
        "/content/drive/MyDrive/BOE/bank_of_england/data/raw/jpmorgan",
        "/content/drive/MyDrive/BOE/bank_of_england/data/raw/ubs"
    ],
    "persist_directory": "/content/drive/MyDrive/BOE/bank_of_england/data/model_outputs",
    "llm_model_name": "google/bigbird-pegasus-large-arxiv",  # For long sequences
    "embedding_model_name": "sentence-transformers/all-mpnet-base-v2",
    "text_generation_pipeline_task": "text2text-generation",
    "max_length": 1024,
    "temperature": 0.1,
    "top_p": 0.8,
    "batch_size": 8,
    "chunk_size": 1000,
    "chunk_overlap": 100,
    "chunk_threshold": 1024,
    "memory_window_k": 10,
    "retriever_search_k": 5
}

# --- Master Agent Prompt (Refined) ---
MASTER_AGENT_PROMPT = (
    "You are an expert in analyzing bank earnings call transcripts. "
    "Your task is to answer questions accurately using provided tools. Be very concise.\n\n"
    "When answering, follow EXACTLY this format:\n\n"
    "Thought: Briefly explain my reasoning. What information do I need? Which tool should I use?\n"
    "Action: Use one of the tools: JP_Morgan_RAG or UBS_RAG. "
    "To use a tool, write: Tool_Name(\"query\"), for example: JP_Morgan_RAG(\"what is net income at JP Morgan?\")."
    "If the question asks about JP Morgan, use JP_Morgan_RAG. If the question asks about UBS, use UBS_RAG.\n"
    "Observation: Report the result from the tool. Keep it concise.\n"
    "Final Answer: Give a final, short answer to the original question.\n\n"
    "Example 1:\n"
    "Question: What is Jamie Dimon's role at JP Morgan?\n"
    "Thought: I need to find information about Jamie Dimon's role at JP Morgan. Since the question is about JP Morgan I should use the JP_Morgan_RAG tool.\n"
    "Action: JP_Morgan_RAG(\"What is Jamie Dimon's role?\")\n"
    "Observation: Jamie Dimon is the CEO of JP Morgan Chase.\n"
    "Final Answer: Jamie Dimon is the CEO of JP Morgan Chase.\n\n"
    "Example 2:\n"
    "Question: What was the operating profit at UBS in Q2 2023?\n"
    "Thought: I need to find the operating profit for UBS in Q2 2023. Since the question is about UBS, I should use the UBS_RAG tool.\n"
    "Action: UBS_RAG(\"What was the operating profit in Q2 2023?\")\n"
    "Observation: UBS's operating profit in Q2 2023 was $1.5 billion.\n"
    "Final Answer: UBS's operating profit in Q2 2023 was $1.5 billion.\n\n"
    "Question: {input}\n"  # Place the question directly in the prompt for clarity
    "Begin!\n" #Include newline
)


# --- RAGChatbot Class ---
class RAGChatbot:
    """
    A RAG chatbot that ingests PDF earnings call transcripts, builds a vector store,
    and uses a ConversationalRetrievalChain for Q&A.
    """
    def __init__(self, config, bank: str): #Added bank parameter for metadata filtering
        self.config = config
        self.hf_token = os.environ.get('HF') # Access token from environment variable
        self.pdf_folders = config["pdf_folders"]
        self.persist_directory = config["persist_directory"]
        self.max_length = config["max_length"]
        self.batch_size = config["batch_size"]
        self.chunk_size = config["chunk_size"]
        self.chunk_overlap = config["chunk_overlap"]
        self.chunk_threshold = config["chunk_threshold"]
        self.memory_window_k = config["memory_window_k"]
        self.retriever_search_k = config["retriever_search_k"]
        self.bank = bank #Added bank attribute

        self._setup_llm()
        self._setup_embeddings()
        self._load_documents()
        self._build_vector_store()
        self._build_summary_index()
        self._setup_retrieval_chain()

    def _setup_llm(self):
        model_name = self.config["llm_model_name"]
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, token=self.hf_token) #Fixed: Use token instead of use_auth_token
        self.model = AutoModelForSeq2SeqLM.from_pretrained(
            model_name, torch_dtype=torch.float16, token=self.hf_token) #Fixed: Use token instead of use_auth_token
        if self.model.config.pad_token_id is None:
            self.model.config.pad_token_id = self.tokenizer.eos_token_id
        if torch.cuda.is_available():
            self.model.to("cuda")
        # Use num_beams=1 to avoid beam search issues that trigger CUDA asserts.
        self.pipe = pipeline(
            self.config["text_generation_pipeline_task"],
            model=self.model,
            tokenizer=self.tokenizer,
            max_length=self.max_length,
            temperature=self.config["temperature"],
            top_p=self.config["top_p"],
            do_sample=True,
            batch_size=self.batch_size,
            num_beams=1,
            pad_token_id=self.tokenizer.eos_token_id
        )
        self.llm = HuggingFacePipeline(pipeline=self.pipe)

    def _setup_embeddings(self):
        emb_model = self.config["embedding_model_name"]
        self.embeddings = HuggingFaceEmbeddings(model_name=emb_model) # Removed token argument

    def _load_documents(self):
        self.documents = []
        for folder in self.pdf_folders:
            bank = os.path.basename(folder).lower()
            files = [f for f in os.listdir(folder) if f.endswith(".pdf")]
            for file in files:
                path = os.path.join(folder, file)
                try:
                    loader = PyPDFLoader(path, extract_images=False)
                    docs = loader.load_and_split()
                    for doc in docs:
                        doc.metadata["bank"] = bank
                        doc.metadata["source_pdf"] = file
                    self.documents.extend(docs)
                    print(f"Loaded: {file} from {folder}")
                except Exception as e:
                    print(f"Error loading {file}: {e}")

    def _chunk_document(self, doc: Document) -> list[Document]:
        tokens = self.tokenizer.encode(doc.page_content)
        if len(tokens) > self.chunk_threshold:
            splitter = RecursiveCharacterTextSplitter(
                chunk_size=self.chunk_size, chunk_overlap=self.chunk_overlap)
            chunks = splitter.split_documents([doc])
            return self._remove_duplicates(chunks)
        return [doc]

    @staticmethod
    def _remove_duplicates(chunks: list[Document]) -> list[Document]:
        seen = set()
        unique = []
        for chunk in chunks:
            text = chunk.page_content.strip()
            if text not in seen:
                seen.add(text)
                unique.append(chunk)
        return unique

    def _build_vector_store(self):
        all_chunks = []
        for doc in self.documents:
            all_chunks.extend(self._chunk_document(doc))
        self.raw_db = Chroma.from_documents(
            all_chunks, embedding=self.embeddings, persist_directory=self.persist_directory)
        self.raw_db.persist()
        print(f"Built raw vector store with {len(all_chunks)} chunks.")

    def _build_summary_index(self):
        # For simplicity, we use the same chunks as summaries.
        all_chunks = []
        for doc in self.documents:
            all_chunks.extend(self._chunk_document(doc))
        self.summary_db = Chroma.from_documents(
            all_chunks, embedding=self.embeddings,
            persist_directory=os.path.join(self.persist_directory, "summaries"))
        self.summary_db.persist()
        print(f"Built summary vector index with {len(all_chunks)} chunks.")

    def _setup_retrieval_chain(self):
        memory = ConversationBufferWindowMemory(
            k=self.memory_window_k, memory_key="chat_history", return_messages=True)
        #Added filter to retriever for bank metadata
        self.retrieval_chain = ConversationalRetrievalChain.from_llm(
            llm=self.llm,
            retriever=self.summary_db.as_retriever(search_kwargs={"k": self.retriever_search_k, "filter": {"bank": self.bank}}),
            memory=memory,
            verbose=True
        )

    def answer_query(self, query: str) -> str:
        response = self.retrieval_chain({"question": query})
        return response.get("answer", "").strip() #Fixed missing return for anser_query

# --- Create Multi-Agent Instances ---
# Filter PDF folders for each bank.
jpm_folders = [folder for folder in CONFIG["pdf_folders"] if "jpmorgan" in folder.lower()]
ubs_folders = [folder for folder in CONFIG["pdf_folders"] if "ubs" in folder.lower()]

# Create separate configurations for each bank.
CONFIG_JPM = CONFIG.copy()
CONFIG_JPM["pdf_folders"] = jpm_folders

CONFIG_UBS = CONFIG.copy()
CONFIG_UBS["pdf_folders"] = ubs_folders

# Initialize separate RAGChatbot instances.
jpm_chatbot = RAGChatbot(CONFIG_JPM, bank="jpmorgan") #Added bank parameter
ubs_chatbot = RAGChatbot(CONFIG_UBS, bank="ubs") #Added bank parameter

# --- Define Tools for Each Agent ---
def jpm_tool(query: str) -> str:
    return jpm_chatbot.answer_query(query)

def ubs_tool(query: str) -> str:
    return ubs_chatbot.answer_query(query)

jpm_tool_instance = Tool(
    name="JP_Morgan_RAG",
    func=jpm_tool,
    description="Answers questions about JP Morgan earnings call transcripts."
)

ubs_tool_instance = Tool(
    name="UBS_RAG",
    func=ubs_tool,
    description="Answers questions about UBS earnings call transcripts."
)

# --- Master Agent Integration ---
master_agent = initialize_agent(
    [jpm_tool_instance, ubs_tool_instance],
    jpm_chatbot.llm,  # Using the same LLM pipeline; both agents use similar config.
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    agent_kwargs={"prefix": MASTER_AGENT_PROMPT} # Using the updated MASTER_AGENT_PROMPT
)

# --- Chatbot Loop (Master Agent) ---
def run_master_agent():
    print("Master Agent Chatbot (type 'exit' to quit)")
    while True:
        user_q = input("You: ")
        if user_q.lower() == "exit":
            print("Exiting Master Agent Chatbot. Goodbye!")
            break
        answer = master_agent.run(user_q)
        print(f"\nMaster Agent Answer:\n{answer}\n")

if __name__ == "__main__":
    run_master_agent()


tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.92M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.51M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/775 [00:00<?, ?B/s]

...

Loaded: 1q23-earnings-transcript.pdf from /content/drive/MyDrive/BOE/bank_of_england/data/raw/jpmorgan
Loaded: 2q23-earnings-transcript.pdf from /content/drive/MyDrive/BOE/bank_of_england/data/raw/jpmorgan
Loaded: 4q24-earnings-transcript.pdf from /content/drive/MyDrive/BOE/bank_of_england/data/raw/jpmorgan
Loaded: jpm-1q24-earnings-call-transcript.pdf from /content/drive/MyDrive/BOE/bank_of_england/data/raw/jpmorgan
Loaded: jpm-2q24-earnings-call-transcript-final.pdf from /content/drive/MyDrive/BOE/bank_of_england/data/raw/jpmorgan
Loaded: jpm-3q23-earnings-call-transcript.pdf from /content/drive/MyDrive/BOE/bank_of_england/data/raw/jpmorgan
Loaded: jpm-4q23-earnings-call-transcript.pdf from /content/drive/MyDrive/BOE/bank_of_england/data/raw/jpmorgan
Loaded: jpmc-third-quarter-2024-earnings-conference-call-transcript.pdf from /content/drive/MyDrive/BOE/bank_of_england/data/raw/jpmorgan
Built raw vector store with 252 chunks.
Built summary vector index with 252 chunks.


Device set to use cuda:0


Loaded: 1q23-earnings-call-remarks.pdf from /content/drive/MyDrive/BOE/bank_of_england/data/raw/ubs
Loaded: 1q24-earnings-call-remarks.pdf from /content/drive/MyDrive/BOE/bank_of_england/data/raw/ubs
Loaded: 2q23-earnings-call-remarks.pdf from /content/drive/MyDrive/BOE/bank_of_england/data/raw/ubs
Loaded: 2q24-earnings-call-remarks.pdf from /content/drive/MyDrive/BOE/bank_of_england/data/raw/ubs
Loaded: 3q23-earnings-call-remarks.pdf from /content/drive/MyDrive/BOE/bank_of_england/data/raw/ubs
Loaded: 3q24-earnings-call-remarks.pdf from /content/drive/MyDrive/BOE/bank_of_england/data/raw/ubs
Loaded: 4q23-earnings-call-remarks.pdf from /content/drive/MyDrive/BOE/bank_of_england/data/raw/ubs
Loaded: 4q24-earnings-call-remarks.pdf from /content/drive/MyDrive/BOE/bank_of_england/data/raw/ubs
Built raw vector store with 228 chunks.
Built summary vector index with 228 chunks.
Master Agent Chatbot (type 'exit' to quit)
You: What was the operating profit at UBS in Q2 2023?


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:695: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `0.8` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.
  warnings.warn(
Attention type 'block_sparse' is not possible if sequence_length: 670 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...




> Entering new AgentExecutor chain...
we show that the set of all possible @xmath0-ary operations of a single entity is a convex set .<n> the set of all possible @xmath0-ary operations of a single entity is a convex set .<n> the set of all possible @xmath0-ary operations of a single entity is a convex set .<n> the definition of the convex set is as follows .<n> * set of all possible @xmath0-ary operations of a single entity * _ introduction .<n> _ the set of all possible @xmath0-ary operations of a single entity is a convex set .<n> the definition of the convex set is as follows .<n> _ let @xmath1 be the set of all possible @xmath0-ary operations of a single entity .<n> _ let @xmath2 be the set of all possible @xmath0-ary operations of a single entity .<n> _ let @xmath3 be the set of all possible @xmath0-ary operations of a single entity .<n> _ let @xmath4 be the set of all possible @xmath0-ary operations of a single entity .<n> _ let @xmath5 be the set of all possible @xmath0-ary op

# Instantiate the chatbot object

# Launch an interactive  Chatbot session

In [ ]:
run_master_agent()